In [ ]:
import xarray as xr
from os.path import isfile, join
from os import listdir
# from google.colab import drive
import torch
import pandas as pd
import numpy as np
import netCDF4 as nc
import datetime
#!pip install geopandas
import geopandas as gpd
# !pip install climata
from climata.usgs import DailyValueIO
from climata.usgs import InstantValueIO
import pandas as pd
import numpy as np
import datetime
from shapely.geometry import Point
from pathlib import Path
# drive.mount('/content/drive', force_remount=True)

In [ ]:
TEST_PERIOD = ("1989-10-01", "1999-09-30")
TRAINING_PERIOD = ("1999-10-01", "2008-09-30")

In [ ]:
def check_if_discharge_file_exists(basin_id, ERA5_discharge_data_folder_name):
    return Path(ERA5_discharge_data_folder_name +'/dis_' + station_id + '.csv').exists()


def check_if_all_percip_files_exist(basin_id, output_folder_name):
    shape_file = output_folder_name + '/shape_' + station_id + '.csv'
    time24_file = output_folder_name + '/time24_' + station_id + '.npy'
    percip24_file = output_folder_name + '/precip24_' + station_id
    timedis24_file = output_folder_name + '/timedis24_' + station_id + '.npy'
    dis24_file = output_folder_name + '/dis24_' + station_id
    info_file = output_folder_name + '/info_' + station_id + '.txt'
    latlon_file = output_folder_name + '/latlon_' + station_id + '.csv'
    list_files = [shape_file, time24_file, percip24_file, timedis24_file, dis24_file, info_file, latlon_file]
    all_files_exist = all(lambda file_path: Paht(file_path).exists() for file_name in list_files)
    return all_files_exist


def parse_single_basin_discharge(station_id, output_folder_name):
    dis_file_exists = check_if_discharge_file_exists(station_id, output_folder_name)
    if dis_file_exists:
      print("The discharge file of the basin: {} exists".format(station_id))
      return
    start_time = pd.to_datetime('1980-01-01 00:00:00')
    end_time = pd.to_datetime('2021-09-15 00:00:00')
    param_id = "00060"
    data = InstantValueIO(
        start_date=start_time,
        end_date=end_time,
        station=station_id,
        parameter=param_id)

    FT2M = 0.3048
    for series in data:
        flow = [r[1] for r in series.data]
        datetimes = [r[0] for r in series.data]

    ls = []
    for i in range(0, len(datetimes)):
      t = datetimes[i].astimezone(datetime.timezone.utc)
      ls.append([t.year,t.month,t.day,t.hour,t.minute,flow[i]*(FT2M**3)])

    df = pd.DataFrame(data=ls, columns=['year','month','day','hour','minute','flow'])
    df_group = df.groupby(by=['year','month','day','hour']).mean()
    df_group = df_group.assign(minute=0)
    df_group.loc[df_group['flow'] < 0 , 'flow'] = np.nan

    filename = output_folder_name +'/dis_' + station_id + '.csv'
    df_group.to_csv(filename)

    
def parse_single_basin_percipitation(station_id, basin_data, discharge_file_name, ERA5_data_folder_name, output_folder_name):
    all_files_exist = check_if_all_percip_files_exist(station_id, output_folder_name)
    if all_files_exist:
      print("all percipitation file of the basin: {} exists".format(station_id))
      return
    basin_boundaries = basin_data.bounds
    # get the minimum and maximum longitude and latitude (square boundaries)
    min_lon = np.squeeze(np.floor((basin_boundaries['minx'].values * 10) / 10))
    min_lat = np.squeeze(np.floor((basin_boundaries['miny'].values * 10) / 10))
    max_lon = np.squeeze(np.ceil((basin_boundaries['maxx'].values * 10) / 10))
    max_lat = np.squeeze(np.ceil((basin_boundaries['maxy'].values * 10) / 10))

    # read the discharge of the required station
    df_dis = pd.read_csv(discharge_file_name)
    # convert the columns of year, month, day, hour, minute to datetime and put it as the dataframe index
    df_dis.index = [datetime.datetime(df_dis['year'][i], df_dis['month'][i],
                                      df_dis['day'][i], df_dis['hour'][i],
                                      df_dis['minute'][i]) for i in range(0, len(df_dis))]
    print(df_dis)
    # read the precipitation of the required station
    # ERA5
    year_start = df_dis['year'].min() - 1
    year_end = df_dis['year'].max()
    print(year_start, year_end)
    list_of_dates = []
    list_of_total_precipitations = []
    for year in range(year_start, year_end + 1):
        print(year)
        fn = f"{ERA5_data_folder_name}/tp_US_{year}.nc"
        dataset = nc.Dataset(fn)
        # ti is an array containing the dates as the number of hours since 1900-01-01 00:00
        # e.g. - [780168, 780169, 780170, ...]
        ti = dataset['time'][:]
        if year == year_start:
            lon = dataset['longitude'][:]
            lat = dataset['latitude'][:]
            max_lon_array = lon.max()
            min_lon_array = lon.min()
            max_lat_array = lat.max()
            min_lat_array = lat.min()
            ind_lon_min = np.squeeze(np.argwhere(lon == max(min_lon, min_lon_array)))
            ind_lon_max = np.squeeze(np.argwhere(lon == min(max_lon, max_lon_array)))
            ind_lat_min = np.squeeze(np.argwhere(lat == max(min_lat, min_lat_array)))
            ind_lat_max = np.squeeze(np.argwhere(lat == min(max_lat, max_lat_array)))
        # multiply by 1000 to convert from meter to mm
        tp = np.asarray(dataset['tp'][:, ind_lat_max:ind_lat_min + 1, ind_lon_min:ind_lon_max + 1]) * 1000
        # convert the time to datetime format and append it to the times array
        times = [datetime.datetime.strptime("1900-01-01 00:00", "%Y-%m-%d %H:%M") + datetime.timedelta(hours=int(ti[i]))
                 for i in range(0, len(ti))]
        times = np.asarray(times)
        list_of_dates.append(times)
        list_of_total_precipitations.append(tp)

    # concatenate the datetimes from all the years
    datetimes = np.concatenate(list_of_dates, axis=0)

    # concatenate the percipitation data from all the years
    precip = np.concatenate(list_of_total_precipitations, axis=0)

    print([station_id, lat[ind_lat_min], lat[ind_lat_max], lon[ind_lon_min],
           lon[ind_lon_max], precip.shape])

    df = pd.DataFrame(data=datetimes, index=datetimes)
    datetimes = df.index.to_pydatetime()

    ls = [[precip[i, :, :]] for i in range(0, len(datetimes))]
    df = pd.DataFrame(data=ls, index=datetimes, columns=['precip'])

    # down sample the datetime data into 1D (1 day) bins and sum the values falling into the same bin
    df1 = df.resample('1D').sum()
    datetimes24 = df1.index.to_pydatetime()
    precip24 = np.stack(df1['precip'].values)

    dis = df_dis['flow'].values
    datetimesdis = df_dis.index.to_pydatetime()

    # downsample the datetime data into 1D (1 day) bins and take the mean
    # of the values falling into the same bin
    df1 = df_dis.resample('1D').mean()
    datetimesdis24 = df1.index.to_pydatetime()
    dis24 = np.stack(df1['flow'].values)

    fn = output_folder_name + '/shape_' + station_id + '.csv'
    pd.DataFrame(data=np.array([(precip24.shape[0],), (precip24.shape[1],), (precip24.shape[2],)]).T,
                 columns=['time', 'lat', 'lon'], index=[1]).to_csv(fn)
    fn = output_folder_name + '/time24_' + station_id + '.npy'
    np.save(fn, datetimes24)
    fn = output_folder_name + '/precip24_' + station_id
    precip24.tofile(fn)
    fn = output_folder_name + '/timedis24_' + station_id + '.npy'
    np.save(fn, datetimesdis24)
    fn = output_folder_name + '/dis24_' + station_id
    dis24.tofile(fn)
    fn = output_folder_name + '/info_' + station_id + '.txt'
    with open(fn, 'w') as f:
        print(precip.shape[0], precip.shape[1], precip.shape[2], lat[ind_lat_min], lat[ind_lat_max], lon[ind_lon_min],
              lon[ind_lon_max], file=f)

    lonb = lon[ind_lon_min:ind_lon_max]
    latb = lat[ind_lat_max:ind_lat_min]
    lslon = [lonb[i] for i in range(0, len(lonb)) for j in range(0, len(latb))]
    lslat = [latb[j] for i in range(0, len(lonb)) for j in range(0, len(latb))]
    lat_lon_lst = []
    for i in range(0, len(lslon)):
        if np.squeeze(basin_data['geometry'].contains(Point(lslon[i], lslat[i]))):
            lat_lon_lst.append([lslat[i], lslon[i]])

    fn = output_folder_name + '/latlon_' + station_id + '.csv'
    pd.DataFrame(data=lat_lon_lst, columns=['lat', 'lon']).to_csv(fn, index=False, float_format='%6.1f')

    fn = output_folder_name + '/info_' + station_id + '.txt'
    with open(fn, 'w') as f:
        print(precip.shape[0], precip.shape[1], precip.shape[2], lat[ind_lat_min], lat[ind_lat_max], lon[ind_lon_min],
              lon[ind_lon_max], file=f)


def main():
    root_folder = "/sci/labs/efratmorin/lab_share/FloodsML/data/ERA5/"
    boundaries_file_name = root_folder + "/HCDN_nhru_final_671.shp"
    ERA5_percip_data_folder_name = root_folder + "/Percipitation/"
    ERA5_discharge_data_folder_name = root_folder + "/Discharge/"
    output_folder_name = root_folder + "/ERA_5_all_data/"
    # read the basins' boundaries file using gpd.read_file()
    basins_data = gpd.read_file(boundaries_file_name)
    for station_id in basins_data["hru_id"]:
      station_id = str(station_id).zfill(8)
      print("working on basin with id: {}".format(station_id))
      parse_single_basin_discharge(station_id, ERA5_discharge_data_folder_name)
      discharge_file_name = ERA5_discharge_data_folder_name + '/dis_' + str(station_id) + '.csv'
      # get the boundaries of the required basin using its station ID
      basin_data = basins_data[basins_data['hru_id'] == int(station_id)]
      parse_single_basin_percipitation(station_id, basin_data, discharge_file_name, ERA5_percip_data_folder_name, output_folder_name)

main()